# 课堂训练

In [1]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [2]:
df = pd.read_excel('weibo.xlsx', index_col = 0)

In [3]:
df.head()

,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域
序号,,,,,,,,
0,#高校通报教师图书馆打电话声音过大出言不逊#公道自在人心，谣言自在人心 ​​,0,0,0,a2331b38901d62d2d9a20529177ef3b3,0,22,湖北
1,转发C,0,0,0,d6dc4470f51fce93cc0cbad8abf55a75,0,33,广西
2,【#刘雨昕运动者联濛#河山覆冰雪，健儿迎冬奥[金牌]全能唱跳不设限，运动联濛开新年�� 期待...,0,0,0,372bc4782eb442b88035f920a7c1a68e,6,85,广东
3,丁程鑫//@丁程鑫后援会官博:#丁程鑫[超话]# ✨#丁程鑫 二十成金筑梦鑫世界# 大年初一...,0,0,0,6fe0d482bd3e78a3483e2a1d57f14ef2,75,1012,广东
4,诶，你们真不要脸诶。。。没资格宣传奥运。。。抵制抵制！,0,0,0,872380d71d6ee9130e8b49d331f2baa9,0,10,广东


## 语料预处理

### 剔除符号与数字

In [4]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['标题/微博内容'][0]
remove_nums(test)[:100]

'高校通报教师图书馆打电话声音过大出言不逊公道自在人心谣言自在人心'

### 分词

In [5]:
# 加载中文停用词词典，可个性化设置
stopwords = open('stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    # words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['标题/微博内容'][0]
clean_text(test)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/kk/ylyfvmrj6zv853wrvp3s_0180000gn/T/jieba.cache
Loading model cost 0.323 seconds.
Prefix dict has been built successfully.


'# 高校 通报 教师 图书馆 打电话 声音 过大 出言不逊 # 公道 自 在 人心 ， 谣言 自 在 人心   \u200b \u200b'

In [6]:
df['标题/微博内容'] = df['标题/微博内容'].astype(str)
df['微博内容分词'] = df['标题/微博内容'].apply(remove_nums)
df['微博内容分词'] = df['微博内容分词'].apply(clean_text)
df['微博内容分词'] = df['微博内容分词'].apply(lambda x: x.split())
df

,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
序号,,,,,,,,,
0,#高校通报教师图书馆打电话声音过大出言不逊#公道自在人心，谣言自在人心 ​​,0,0,0,a2331b38901d62d2d9a20529177ef3b3,0,22,湖北,"[高校, 通报, 教师, 图书馆, 打电话, 声音, 过大, 出言不逊, 公道, 自, 在,..."
1,转发C,0,0,0,d6dc4470f51fce93cc0cbad8abf55a75,0,33,广西,[转发]
2,【#刘雨昕运动者联濛#河山覆冰雪，健儿迎冬奥[金牌]全能唱跳不设限，运动联濛开新年�� 期待...,0,0,0,372bc4782eb442b88035f920a7c1a68e,6,85,广东,"[刘雨昕, 运动, 者, 联, 濛, 河山, 覆, 冰雪, 健儿, 迎, 冬奥, 金牌, 全..."
3,丁程鑫//@丁程鑫后援会官博:#丁程鑫[超话]# ✨#丁程鑫 二十成金筑梦鑫世界# 大年初一...,0,0,0,6fe0d482bd3e78a3483e2a1d57f14ef2,75,1012,广东,"[丁程鑫, 丁程鑫, 后援会, 官博丁, 程鑫, 超话, 丁程鑫, 二十, 成金筑梦鑫, 世..."
4,诶，你们真不要脸诶。。。没资格宣传奥运。。。抵制抵制！,0,0,0,872380d71d6ee9130e8b49d331f2baa9,0,10,广东,"[诶, 你们, 真, 不要脸, 诶, 没, 资格, 宣传, 奥运, 抵制, 抵制]"
...,...,...,...,...,...,...,...,...,...
96,转发C,0,0,0,66431bf833123a929242c248ff47955c,531,356,广东,[转发]
97,中国代表团已经实现了单届冬奥会金牌、奖牌最多的愿望，北京冬奥会已经成为历史上最好成绩的一届冬...,0,0,0,c7f2ee057de17cec471766f921a116a2,7550,1795,广东,"[中国, 代表团, 已经, 实现, 了, 单届, 冬奥会, 金牌, 奖牌, 最多, 的, 愿..."
98,//@文轩星系:感谢@北京台春晚 分享。尽享中华之美，共祝冬奥盛会。和@时代少年团-宋亚轩 ...,0,0,0,18ec3618f9fc5c0b6064ee1ffa7feb59,13,92,山西,"[文轩, 星系, 感谢, 北京, 台, 春晚, 分享, 尽享, 中华, 之美, 共祝, 冬奥..."


## LDA主题模型

In [7]:
dictionary = corpora.Dictionary(df['微博内容分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['微博内容分词']]     # 根据分词结果创建语料库

In [8]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [9]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.013*"我" + 0.013*"害羞" + 0.009*"好" + 0.009*"冬奥会" + 0.009*"的"')
(1, '0.021*"的" + 0.017*"北京" + 0.016*"宋亚轩" + 0.012*"和" + 0.012*"了"')
(2, '0.077*"转发" + 0.022*"少年" + 0.019*"冬奥" + 0.017*"时代" + 0.015*"蔡"')
(3, '0.031*"的" + 0.024*"你" + 0.020*"濛" + 0.019*"刘雨昕" + 0.018*"运动"')
(4, '0.041*"一起" + 0.026*"向" + 0.026*"未来" + 0.026*"冬奥" + 0.013*"的"')


In [10]:
df['微博内容分词'].iloc[0]

['高校',
 '通报',
 '教师',
 '图书馆',
 '打电话',
 '声音',
 '过大',
 '出言不逊',
 '公道',
 '自',
 '在',
 '人心',
 '谣言',
 '自',
 '在',
 '人心']

In [11]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9528998732566833	 
Topic: 0.031*"的" + 0.024*"你" + 0.020*"濛" + 0.019*"刘雨昕" + 0.018*"运动" + 0.018*"联" + 0.015*"者" + 0.014*"我" + 0.012*"冬奥" + 0.011*"一起"

Score: 0.01178612932562828	 
Topic: 0.041*"一起" + 0.026*"向" + 0.026*"未来" + 0.026*"冬奥" + 0.013*"的" + 0.013*"为" + 0.013*"肖战" + 0.011*"周深" + 0.011*"和" + 0.011*"唱响"

Score: 0.011775565333664417	 
Topic: 0.077*"转发" + 0.022*"少年" + 0.019*"冬奥" + 0.017*"时代" + 0.015*"蔡" + 0.015*"徐坤" + 0.015*"团" + 0.012*"北京" + 0.010*"和" + 0.010*"一起"

Score: 0.011770505458116531	 
Topic: 0.013*"我" + 0.013*"害羞" + 0.009*"好" + 0.009*"冬奥会" + 0.009*"的" + 0.009*"墩" + 0.009*"为" + 0.009*"前辈" + 0.009*"萧薰儿" + 0.009*"保护"

Score: 0.011767969466745853	 
Topic: 0.021*"的" + 0.017*"北京" + 0.016*"宋亚轩" + 0.012*"和" + 0.012*"了" + 0.011*"加油" + 0.011*"冬奥" + 0.010*"冰雪" + 0.010*"少年" + 0.010*"刘雨昕"


In [12]:
documents = df['微博内容分词'].values

In [13]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(0, 0.0117705045), (1, 0.011767969), (2, 0.011775569), (3, 0.95289975), (4, 0.011786178)]

Document 2:
[(0, 0.1000011), (1, 0.1000006), (2, 0.5999103), (3, 0.10000045), (4, 0.10008755)]

Document 3:
[(3, 0.97932184)]

Document 4:
[(1, 0.9712015)]

Document 5:
[(0, 0.01667134), (1, 0.016669229), (2, 0.93330055), (3, 0.016689079), (4, 0.01666978)]

Document 6:
[(0, 0.1000011), (1, 0.1000006), (2, 0.5999103), (3, 0.10000045), (4, 0.10008756)]

Document 7:
[(0, 0.9816402)]

Document 8:
[(0, 0.1000011), (1, 0.1000006), (2, 0.59991014), (3, 0.10000045), (4, 0.10008772)]

Document 9:
[(1, 0.96314347)]

Document 10:
[(0, 0.018198645), (1, 0.018253349), (2, 0.018249705), (3, 0.01826833), (4, 0.9270299)]



### 可视化

In [14]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) # 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
# 显示可视化界面
pyLDAvis.display(lda_vis)

# 官方双减数据

In [15]:
df = pd.read_excel('官方双减数据.xlsx', index_col = 0).astype(str)

In [16]:
df.head()

,标题,内容
序号,,
1,中共中央办公厅 国务院办公厅印发《关于进一步减轻义务教育阶段学生作业负担和校外培训负担的意见》,近日，中共中央办公厅、国务院办公厅印发了《关于进一步减轻义务教育阶段学生作业负担和校外培训负...
2,坚决贯彻中央决策部署 深入推进“双减”工作,2021年5月21日，中共中央总书记、国家主席、中央军委主席习近平主持召开中央全面深化改革委...
3,“双减”目标如何实现,近日，中共中央办公厅、国务院办公厅印发了《关于进一步减轻义务教育阶段学生作业负担和校外培训负...
4,以人民为中心，将“双减”落到实处,校内作业负担重，校外培训负担更重，师生家长苦其久矣。近日，中共中央办公厅、国务院办公厅印发《...
5,nan,近日，中共中央办公厅、国务院办公厅印发《关于进一步减轻义务教育阶段学生作业负担和校外培训负担...


## 语料预处理

### 剔除符号与数字

In [17]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['内容'][8]
remove_nums(test)[:100]

'习近平总书记强调义务教育是国民教育的重中之重要全面贯彻党的教育方针落实立德树人根本任务充分发挥学校教书育人主体功能强化线上线下校外培训机构规范管理一年前中办国办印发关于进一步减轻义务教育阶段学生作业负'

### 分词

In [17]:
# 加载中文停用词词典，可个性化设置
stopwords = open('stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    # words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['内容'][8]
clean_text(test)[:20]

'习近平 总书记 强调 ， 义务教育 是 '

In [18]:
df['内容'] = df['内容'].astype(str)
df['内容分词'] = df['内容'].apply(remove_nums)
df['内容分词'] = df['内容分词'].apply(clean_text)
df['内容分词'] = df['内容分词'].apply(lambda x: x.split())
df

,标题,内容,内容分词
序号,,,
1,中共中央办公厅 国务院办公厅印发《关于进一步减轻义务教育阶段学生作业负担和校外培训负担的意见》,近日，中共中央办公厅、国务院办公厅印发了《关于进一步减轻义务教育阶段学生作业负担和校外培训负...,"[近日, 中共中央办公厅, 国务院办公厅, 印发, 了, 关于, 进一步, 减轻, 义务教育..."
2,坚决贯彻中央决策部署 深入推进“双减”工作,2021年5月21日，中共中央总书记、国家主席、中央军委主席习近平主持召开中央全面深化改革委...,"[年月日, 中共中央, 总书记, 国家, 主席, 中央军委, 主席, 习近平, 主持, 召开..."
3,“双减”目标如何实现,近日，中共中央办公厅、国务院办公厅印发了《关于进一步减轻义务教育阶段学生作业负担和校外培训负...,"[近日, 中共中央办公厅, 国务院办公厅, 印发, 了, 关于, 进一步, 减轻, 义务教育..."
4,以人民为中心，将“双减”落到实处,校内作业负担重，校外培训负担更重，师生家长苦其久矣。近日，中共中央办公厅、国务院办公厅印发《...,"[校内, 作业负担, 重, 校外, 培训, 负担, 更重, 师生, 家长, 苦其久, 矣, ..."
5,nan,近日，中共中央办公厅、国务院办公厅印发《关于进一步减轻义务教育阶段学生作业负担和校外培训负担...,"[近日, 中共中央办公厅, 国务院办公厅, 印发, 关于, 进一步, 减轻, 义务教育, 阶..."
...,...,...,...
441,教育部部署2024年教育工作重点任务,新华社北京1月11日电（记者施雨岑、徐壮）记者从11日举行的2024年全国教育工作会议上了解...,"[新华社, 北京, 月, 日电, 记者, 施雨岑, 徐壮, 记者, 从, 日, 举行, 的,..."
442,教育部召开2024年度基础教育重点工作部署会,近期，教育部在贵州省贵阳市召开2024年度基础教育重点工作部署会，教育部党组成员、副部长、总...,"[近期, 教育部, 在, 贵州省, 贵阳市, 召开, 年度, 基础教育, 重点, 工作, 部..."
443,教育部召开2024年教育督导重点工作部署会暨义务教育优质均衡发展督导评估工作推进会,近日，教育部在江苏南京召开2024年教育督导重点工作部署会暨义务教育优质均衡发展督导评估工作...,"[近日, 教育部, 在, 江苏, 南京, 召开, 年, 教育, 督导, 重点, 工作, 部署..."


## LDA主题模型

In [19]:
dictionary = corpora.Dictionary(df['内容分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['内容分词']]     # 根据分词结果创建语料库

In [20]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [21]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.061*"的" + 0.015*"了" + 0.014*"学生" + 0.012*"在" + 0.010*"和"')
(1, '0.036*"的" + 0.017*"学生" + 0.015*"学校" + 0.013*"和" + 0.013*"课后"')
(2, '0.043*"培训" + 0.035*"的" + 0.023*"校外" + 0.019*"机构" + 0.014*"和"')
(3, '0.025*"的" + 0.018*"教育" + 0.013*"和" + 0.009*"了" + 0.009*"等"')
(4, '0.021*"的" + 0.009*"教育" + 0.006*"和" + 0.006*"义务教育" + 0.005*"发展"')


In [22]:
df['内容分词'].iloc[0][:20]

['近日',
 '中共中央办公厅',
 '国务院办公厅',
 '印发',
 '了',
 '关于',
 '进一步',
 '减轻',
 '义务教育',
 '阶段',
 '学生',
 '作业负担',
 '和',
 '校外',
 '培训',
 '负担',
 '的',
 '意见',
 '并',
 '发出通知']

In [23]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5087704658508301	 
Topic: 0.043*"培训" + 0.035*"的" + 0.023*"校外" + 0.019*"机构" + 0.014*"和" + 0.010*"等" + 0.010*"学科" + 0.009*"工作" + 0.009*"类" + 0.009*"教育"

Score: 0.49100416898727417	 
Topic: 0.036*"的" + 0.017*"学生" + 0.015*"学校" + 0.013*"和" + 0.013*"课后" + 0.012*"教育" + 0.012*"作业" + 0.012*"服务" + 0.010*"双减" + 0.010*"工作"


In [24]:
documents1 = df['内容分词'].values

In [25]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document1 {i+1}:")
    print(doc_topics)
    print()

Document1 1:
[(0, 0.016005587), (1, 0.93590504), (2, 0.015670346), (3, 0.016906338), (4, 0.015512657)]

Document1 2:
[(0, 0.10037554), (1, 0.59844655), (2, 0.10037246), (3, 0.10037972), (4, 0.100425705)]

Document1 3:
[(0, 0.97027314)]

Document1 4:
[(0, 0.3456153), (1, 0.26142105), (2, 0.11850001), (3, 0.25717577), (4, 0.01728789)]

Document1 5:
[(0, 0.19582826), (1, 0.022644566), (2, 0.6032095), (3, 0.15595897), (4, 0.022358716)]

Document1 6:
[(0, 0.10037557), (1, 0.59844637), (2, 0.10037249), (3, 0.10037975), (4, 0.10042578)]

Document1 7:
[(0, 0.60636586), (1, 0.19868723), (2, 0.09329279), (4, 0.09551484)]

Document1 8:
[(0, 0.10037556), (1, 0.5984465), (2, 0.10037248), (3, 0.10037974), (4, 0.10042574)]

Document1 9:
[(0, 0.87259954), (1, 0.011953968), (2, 0.01185402), (3, 0.091367625), (4, 0.01222481)]

Document1 10:
[(0, 0.8831482), (1, 0.029102925), (2, 0.029100895), (3, 0.029463897), (4, 0.02918415)]



In [26]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) # 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
# 显示可视化界面
pyLDAvis.display(lda_vis)

# 民间双减数据

In [27]:
df = pd.read_excel('民间双减数据.xlsx', index_col = 0).astype(str)

In [28]:
df.head()

,标题链接,m-box-col,头像,weibo-text,time,m-ctrl-box,m-ctrl-box1,m-ctrl-box2,from,m-icon,缩略图
标题,,,,,,,,,,,
"小土大橙子,#双减#","javascript:void(0),javascript:;",#双减#0讨论 0阅读,https://tvax1.sinaimg.cn/crop.0.0.640.640.180/...,该说不说，最该减的就是语文吧。答案又不确定，考那么难是要干啥，课本一滴水考试一片海，不明确就...,1-18 18:45,59,237,1573,来自 iPhone 12,https://h5.sinaimg.cn/upload/108/1866/2022/11/...,https://wx3.sinaimg.cn/thumb180/0024cZx9ly8hgy...
美杜莎麻麻,javascript:void(0),2,https://tvax4.sinaimg.cn/crop.0.0.1080.1080.18...,现在小学写得少，还有个减负双减的原因。 不过感觉今年好像没减那样厉害了，这学期有作业有单元测...,1-4 11:29,2,92,88,来自 nova8 我由我掌镜,https://h5.sinaimg.cn/upload/108/1866/2022/11/...,nan
晏凌羊,javascript:void(0),3,https://tva2.sinaimg.cn/crop.0.0.750.750.180/5...,网友私信（后附胖羊回复）： 羊羊好，匿名公开咨询。 非常同意你写的”生活在这片土地上，不...,1-25 00:05,3,6,64,来自 微博网页版,https://h5.sinaimg.cn/upload/108/1866/2022/11/...,nan
"初见旧城以西,#长安区教育局#","javascript:void(0),javascript:;",#长安区教育局#0讨论 0阅读,https://tvax1.sinaimg.cn/crop.0.0.1058.1058.18...,#长安区教育局##兴国中学##双减政策# 西安市长安区兴国中学提前开学，无视国家的寒假放假计...,2-19 16:28,18,16,83,来自 iPhone客户端,nan,https://wx2.sinaimg.cn/thumb180/0024cZx9ly8gyd...
铲你鸡哇,javascript:void(0),转发,https://tvax3.sinaimg.cn/crop.0.0.1080.1080.18...,苏新航你再这么黏我可就诅咒重庆中小学双减政策取消了啊！,2023-8-21,转发,7,345,nan,https://h5.sinaimg.cn/upload/108/1866/2022/11/...,nan


## 语料预处理

### 剔除符号与数字

In [29]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['weibo-text'][0]
remove_nums(test)[:100]

'该说不说最该减的就是语文吧答案又不确定考那么难是要干啥课本一滴水考试一片海不明确就是造成恐慌和疯狂内卷的因素之一双减'

### 分词

In [30]:
# 加载中文停用词词典，可个性化设置
stopwords = open('stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    # words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['weibo-text'][0]
clean_text(test)

'该 说 不 说 ， 最 该减 的 就是 语文 吧 。 答案 又 不 确定 ， 考 那么 难是 要 干 啥 ， 课本 一滴水 考试 一片 海 ， 不 明确 就是 造成 恐慌 和 疯狂 内卷 的 因素 之一   # 双减 #'

In [31]:
df['weibo-text'] = df['weibo-text'].astype(str)
df['weibo-text分词'] = df['weibo-text'].apply(remove_nums)
df['weibo-text分词'] = df['weibo-text分词'].apply(clean_text)
df['weibo-text分词'] = df['weibo-text分词'].apply(lambda x: x.split())
df

,标题链接,m-box-col,头像,weibo-text,time,m-ctrl-box,m-ctrl-box1,m-ctrl-box2,from,m-icon,缩略图,weibo-text分词
标题,,,,,,,,,,,,
"小土大橙子,#双减#","javascript:void(0),javascript:;",#双减#0讨论 0阅读,https://tvax1.sinaimg.cn/crop.0.0.640.640.180/...,该说不说，最该减的就是语文吧。答案又不确定，考那么难是要干啥，课本一滴水考试一片海，不明确就...,1-18 18:45,59,237,1573,来自 iPhone 12,https://h5.sinaimg.cn/upload/108/1866/2022/11/...,https://wx3.sinaimg.cn/thumb180/0024cZx9ly8hgy...,"[该, 说, 不, 说, 最, 该减, 的, 就是, 语文, 吧, 答案, 又, 不, 确定..."
美杜莎麻麻,javascript:void(0),2,https://tvax4.sinaimg.cn/crop.0.0.1080.1080.18...,现在小学写得少，还有个减负双减的原因。 不过感觉今年好像没减那样厉害了，这学期有作业有单元测...,1-4 11:29,2,92,88,来自 nova8 我由我掌镜,https://h5.sinaimg.cn/upload/108/1866/2022/11/...,nan,"[现在, 小学, 写得少, 还有, 个, 减负, 双减, 的, 原因, 不过, 感觉, 今年..."
晏凌羊,javascript:void(0),3,https://tva2.sinaimg.cn/crop.0.0.750.750.180/5...,网友私信（后附胖羊回复）： 羊羊好，匿名公开咨询。 非常同意你写的”生活在这片土地上，不...,1-25 00:05,3,6,64,来自 微博网页版,https://h5.sinaimg.cn/upload/108/1866/2022/11/...,nan,"[网友, 私信, 后, 附胖羊, 回复, 羊羊, 好, 匿名, 公开, 咨询, 非常, 同意..."
"初见旧城以西,#长安区教育局#","javascript:void(0),javascript:;",#长安区教育局#0讨论 0阅读,https://tvax1.sinaimg.cn/crop.0.0.1058.1058.18...,#长安区教育局##兴国中学##双减政策# 西安市长安区兴国中学提前开学，无视国家的寒假放假计...,2-19 16:28,18,16,83,来自 iPhone客户端,nan,https://wx2.sinaimg.cn/thumb180/0024cZx9ly8gyd...,"[长安区, 教育局, 兴国, 中学, 双减, 政策, 西安市, 长安区, 兴国, 中学, 提..."
铲你鸡哇,javascript:void(0),转发,https://tvax3.sinaimg.cn/crop.0.0.1080.1080.18...,苏新航你再这么黏我可就诅咒重庆中小学双减政策取消了啊！,2023-8-21,转发,7,345,nan,https://h5.sinaimg.cn/upload/108/1866/2022/11/...,nan,"[苏, 新航, 你, 再, 这么, 黏, 我, 可, 就, 诅咒, 重庆, 中小学, 双减,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
中国教育新闻网,javascript:void(0),转发,https://tvax2.sinaimg.cn/crop.3.10.188.188.180...,【从最薄弱处入手】“太空丝瓜种子和普通的丝瓜种子一样吗？”近日，在山东省菏泽市牡丹区都司镇白...,4-8 09:34,转发,评论,赞,来自 微博网页版,https://h5.sinaimg.cn/upload/108/1866/2022/11/...,nan,"[从, 最, 薄弱, 处, 入手, 太空, 丝瓜, 种子, 和, 普通, 的, 丝瓜, 种子..."
蔓越小莓,javascript:void(0),转发,https://tvax1.sinaimg.cn/crop.0.0.512.512.180/...,今天K3要去参观隔壁的小学咯！去看看你不会过的那种生活这个片区的公立可卷了，乐乐那批没赶上双...,4-8 08:17,转发,评论,赞,来自 iPhone 15 Pro Max,https://h5.sinaimg.cn/upload/108/1866/2022/11/...,https://wx2.sinaimg.cn/orj360/78417387gy1hoj3v...,"[今天, 要, 去, 参观, 隔壁, 的, 小学, 咯去, 看看, 你, 不会, 过, 的,..."
陕西都市热线,javascript:void(0),转发,https://tvax1.sinaimg.cn/crop.0.0.512.512.180/...,【西安市教育局通报违规校外培训行为】西安市碑林区、长安区、高陵区严格落实《校外培训行政处罚暂...,4-7 18:11,转发,评论,赞,来自 微博网页版,https://h5.sinaimg.cn/upload/108/1866/2022/11/...,https://wx2.sinaimg.cn/orj360/6d146ea8ly1hoifk...,"[西安市, 教育局, 通报, 违规, 校外, 培训, 行为, 西安市, 碑林区, 长安区, ..."


## LDA主题模型

In [32]:
dictionary = corpora.Dictionary(df['weibo-text分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['weibo-text分词']]     # 根据分词结果创建语料库

In [33]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [34]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.090*"的" + 0.053*"中等生" + 0.032*"双减" + 0.028*"一个" + 0.027*"下"')
(1, '0.047*"的" + 0.023*"假期" + 0.023*"静海" + 0.016*"自己" + 0.016*"工作"')
(2, '0.069*"校外" + 0.061*"培训" + 0.052*"西安市" + 0.035*"违规" + 0.035*"行为"')
(3, '0.059*"的" + 0.058*"种子" + 0.044*"在" + 0.044*"牡丹区" + 0.030*"和"')
(4, '0.068*"的" + 0.034*"家长" + 0.034*"还是" + 0.027*"幼儿园" + 0.021*"不"')


In [35]:
df['weibo-text分词'].iloc[0][:20]

['该',
 '说',
 '不',
 '说',
 '最',
 '该减',
 '的',
 '就是',
 '语文',
 '吧',
 '答案',
 '又',
 '不',
 '确定',
 '考',
 '那么',
 '难是',
 '要',
 '干',
 '啥']

In [36]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.7294279932975769	 
Topic: 0.090*"的" + 0.053*"中等生" + 0.032*"双减" + 0.028*"一个" + 0.027*"下" + 0.027*"家庭" + 0.027*"背景" + 0.027*"城市" + 0.027*"选择" + 0.017*"是"

Score: 0.21617178618907928	 
Topic: 0.068*"的" + 0.034*"家长" + 0.034*"还是" + 0.027*"幼儿园" + 0.021*"不" + 0.021*"双减" + 0.020*"是" + 0.020*"孩子" + 0.020*"该" + 0.014*"你"

Score: 0.04359707608819008	 
Topic: 0.059*"的" + 0.058*"种子" + 0.044*"在" + 0.044*"牡丹区" + 0.030*"和" + 0.030*"科学" + 0.029*"菏泽市" + 0.029*"普通" + 0.029*"太空" + 0.029*"丝瓜"


In [37]:
documents2 = df['weibo-text分词'].values

In [38]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document2 {i+1}:")
    print(doc_topics)
    print()

Document2 1:
[(0, 0.80647236), (1, 0.021058246), (2, 0.12863936), (3, 0.021486659), (4, 0.022343341)]

Document2 2:
[(0, 0.59874), (1, 0.100313604), (2, 0.100314334), (3, 0.10031924), (4, 0.10031283)]

Document2 3:
[(0, 0.72714204), (1, 0.1578051), (2, 0.014341515), (3, 0.014565827), (4, 0.08614551)]

Document2 4:
[(0, 0.6445676), (1, 0.020667093), (2, 0.020412244), (3, 0.021669159), (4, 0.29268396)]

Document2 5:
[(0, 0.81815416), (1, 0.121346876), (2, 0.020165844), (3, 0.020167938), (4, 0.02016519)]

Document2 6:
[(0, 0.59874004), (1, 0.1003136), (2, 0.10031433), (3, 0.10031922), (4, 0.10031282)]

Document2 7:
[(0, 0.6933979), (1, 0.08442358), (3, 0.04244212), (4, 0.17125143)]

Document2 8:
[(0, 0.59874), (1, 0.100313604), (2, 0.100314334), (3, 0.10031924), (4, 0.10031283)]

Document2 9:
[(0, 0.7898438), (1, 0.022761207), (2, 0.022275705), (3, 0.022320293), (4, 0.142799)]

Document2 10:
[(0, 0.50684196), (1, 0.19567902), (2, 0.028624346), (3, 0.24011002), (4, 0.028744657)]



### 可视化

In [39]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) # 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
# 显示可视化界面
pyLDAvis.display(lda_vis)